In [ ]:
import cv2
import numpy as np
import face_recognition
import pandas as pd
from datetime import datetime
from joblib import load
from tensorflow.keras.models import model_from_json
from config import FACE_ID_MODEL, LABEL_ENCODER, ATTENDANCE_FILE, START_TIME, END_TIME

# Load face recognition model
clf = load(FACE_ID_MODEL)
le = load(LABEL_ENCODER)

# Load emotion model
with open("models/emotion_model_arch.json", "r") as f:
    emotion_model = model_from_json(f.read())
emotion_model.load_weights("models/emotion_model_weights.h5")

EMOTIONS = ["Angry","Disgust","Fear","Happy","Sad","Surprise","Neutral"]

# Attendance dict
attendance = {}

def within_time_window():
    now = datetime.now().strftime("%H:%M")
    return START_TIME <= now <= END_TIME

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes = face_recognition.face_locations(rgb, model="hog")
    encodings = face_recognition.face_encodings(rgb, boxes)

    for enc, box in zip(encodings, boxes):
        preds = clf.predict([enc])
        student_name = le.inverse_transform(preds)[0]

        top, right, bottom, left = box
        face_img = frame[top:bottom, left:right]
        gray_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        gray_face = cv2.resize(gray_face, (48,48))
        gray_face = gray_face.astype("float")/255.0
        gray_face = np.expand_dims(gray_face, axis=(0,-1))

        emotion_pred = emotion_model.predict(gray_face)
        emotion_label = EMOTIONS[np.argmax(emotion_pred)]

        if within_time_window():
            attendance[student_name] = {
                "time": datetime.now().strftime("%H:%M:%S"),
                "emotion": emotion_label,
                "status": "Present"
            }

        cv2.rectangle(frame, (left, top), (right, bottom), (0,255,0), 2)
        cv2.putText(frame, f"{student_name} - {emotion_label}", (left, top-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    cv2.imshow("Attendance System", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

# Save attendance
if attendance:
    df = pd.DataFrame.from_dict(attendance, orient="index")
    df.to_csv(ATTENDANCE_FILE, index_label="Student")
    print(f"[OK] Attendance saved to {ATTENDANCE_FILE}")
else:
    print("[INFO] No attendance recorded.")
